# Hazard assessment for drought

Click [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CLIMAAX/DROUGHTS/main?labpath=DROUGHTS_notebook_1.ipynb) to launch this workflow on MyBinder. 

### Quantifying drought hazard

Hazard indicators Drought hazard (dH) for a given region is estimated as the probability of exceedance of the median of regional (e.g., EU level) severe precipitation deficits for the historical reference period 1979 -2019.
The precipitation deficit is calculated using the weighted anomaly of standardized precipitation (WASP) index. This index accounts for precipitation seasonal patterns and is computed by summing weighted and standardized monthly precipitation anomalies $^1$.

We use the weighted anomaly of standardized precipitation (WASP) index to define the severity of precipitation deficit. The WASP-index takes into account the annual seasonality of the precipitation cycle and is computed by summing weighted standardized monthly precipitation anomalies (see Eq. 1). Where $P_{n,m}$ is each region's monthly precipitation, $T_m$ is a monthly treshold defining precipitation severity, and $T_A$ is an annual threshold for precipitation severity. The thresholds are defined by dividing multi-annual monthly observed rain using the 'Fisher-jenks' classigication algorithm $^2$. 

Eq. 1: $$WASP_j = \Sigma_{P_{n,m} < T_m}^{P_{n,m} >= T_m}( \frac{P_{n,m} - T_m}{T_m})*\frac{T_m}{T_A}$$

#### Hazard data and methods:

This algorithm requires monthly total precipitation for each NUTS2 region during the historical reference period. Usually, these are observation-based or simulated time series of gridded precipitation data. Processing these data is performed by applying Geographic Information System (GIS) techniques, to extract an aggregated value (e.g., total precipitation) of the data points located within each area of interest (e.g., NUTS2 region). Zonal statistics is widely used for that purpose.

Point, observation-based datasets are an alternative data source, usually collected by meteorological station networks. One can choose the data collected in one or more (e.g., average) representative stations per area of interest to construct a NUTS2 level dataset. The algorithm expects a table in which each row represnt a month-year combination, and each column an area of interest. The first column represnts the date following this format YYYY-MM-DD. The **title of the first columns has to be 'timing', and the rest of the titles has to be the codes of the areas of interest (e.g., NUTS2), which have to be identical to the codes as they appear in the NUTS2 spatial data from the [European Commision](https://ec.europa.eu/eurostat/en/web/nuts/background)**.


# Workflow implementation

### Load libraries

In this notebook we will use the following Python libraries:
- [os](https://docs.python.org/3/library/os.html) - To create directories and work with files
- [urllib](https://docs.python.org/3/library/urllib.html) - To access to online resources
- [pandas](https://pandas.pydata.org/docs/user_guide/index.html) - To create and manage data frames (tables) in Python
- [geopandas](https://geopandas.org/en/stable/docs.html) - Extend pandas to store and manipulate spatial data
- [numpy](https://numpy.org/doc/stable/) - For basic math tools and operations
- [scipy](https://scipy.org/) - Provide advanced mathematical tools and optimization capacities 
- [jenkspy](https://github.com/mthh/jenkspy) - To apply Fisher-Jenks alogrithm 
- [json](https://docs.python.org/3/library/json.html) - To load, store and manipuilate JSON objects
- [pyproj](https://pyproj4.github.io/pyproj/stable/) - An interface to a geographic projections and transformations library
- [matplotlib](https://matplotlib.org/) - For plotting
- [plotly](https://plotly.com/python/) - For dynamic and interactive plotting
- [datetime](https://docs.python.org/3/library/datetime.html) - For handling dates in Python

In [18]:
# lOAD LIBRARIES
import os
import urllib
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy
import jenkspy
import json
import pyproj
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime

# Function for calculating drought hazard indices
%run DROUGHTS_functions.ipynb

### Define working environment and global parameters
This workflow relies on pre-proceessed data. The user will define the path to the data folder and the code below would create a folder for outputs.


In [19]:
# Set Global Settings

workflow_folder = './sample_data/'

# debug if folder does not exist - issue an error to check path

# create outputs folder
if not os.path.exists(os.path.join(workflow_folder, 'outputs')):
    os.makedirs(os.path.join(workflow_folder, 'outputs'))

# set country = 0 to map all Europe
country = 0

# alternatively choose one country code (2-digits), e.g. 'IT' for Italy, or more than one to choose several country
# e.g., run for North Macedonia and Greece, by setting country = ['MK', 'EL']

# Filter countries/locations with missing data (e.g., Liechtenstein, Malta, ..), or very small/remote islands.
filterNUTS = ['NO0B', 'LI00', 'MT00', 'FRY2', 'FRY5', 'ES63', 'ES64', 'IS00',\
             'FRY1', 'FRY3', 'FRY4', 'PT20', 'PT30', 'ES70']

# Filter non-European countries (+ missing data): Turkey, England
filterCNTS = ['UK', 'TR']

# load nuts2 spatial data
print('Load NUTS2 map with three sample regions')
json_nuts_path = 'https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_01M_2021_4326_LEVL_2.geojson'
nuts = load_nuts_json(json_nuts_path)
nuts = nuts.query('NUTS_ID not in @filterNUTS')
nuts = nuts.query('CNTR_CODE not in @filterCNTS')
# Select 3 Polygons in IT to use as an ouput example - TEMPORARY FOR THE DEMO

if country == 0:
    regions = list(nuts['NUTS_ID'])
else:
    regions = list(nuts.loc[np.isin(nuts['CNTR_CODE'], country), ]['NUTS_ID'])

# define output table 
nuts = nuts.loc[nuts['NUTS_ID'].isin(regions)]
output = pd.DataFrame(regions, columns = ['NUTS_ID'])
print('>>>>> NUTS2 map has loaded succefully.')


Load NUTS2 map with three sample regions
>>>>> NUTS2 map has loaded succefully.


### Loading precipitation data and calculate hazard indices

In [24]:
# Load precipitation data
print("Analyzing drought hazard. This process may take few minutes...")
print('\n')
precip = pd.read_csv(os.path.join(workflow_folder, "drought_hazard.csv"))
# convert timing column to datetime
precip['timing'] = pd.to_datetime(precip['timing'], format = '%Y-%m-%d') 
#'%b-%Y'

# column subset based on selected regions
col_subset = np.isin(precip.columns, regions)
col_subset[0] = True 
precip = precip.loc[:, col_subset]

# print head of the table
print('Input precipitation data (top 3 rows): ')
print(precip.head(3))

print('\n')

## calculate WASP Index (Weighted Anomaly Standardized Precipitation) monthly threshold 

# empty arrays and tables for intermediate and final results
WASP = []
WASP_global = []
drought_class = precip.copy()

# prepare output for drought event index - WASP_j- list of lists wasp = [[rid1], [rid2], ...]
for i in range(1, len(precip.columns)):
    # For every NUTS2 out of all regions - do the following:
    
    # empty array for the monthly water deficit thresholds
    t_m = []
    for mon_ in range(1, 13):
        # For every month out of all all months (January, ..., December) - do the following:
        
        # calculcate monthly dropught threshold -\
            # using a division of the data into to clusters with the Jenks' (Natural breaks) algorithm
        r_idx = precip.index[precip.timing.dt.month == mon_].tolist()
        t_m_last = jenkspy.jenks_breaks(precip.iloc[r_idx, i], n_classes = 2)[1]
        t_m.append(t_m_last)
        
        # Define every month with water deficity (precipitation < threshold) as a drought month
        drought_class.iloc[r_idx, i] = (drought_class.iloc[r_idx, i] < t_m_last).astype(int)

    # calculate annual water deficit threshold
    t_a = sum(t_m)
    
    # calculate droughts' magnitude and duration using the WASP indicator
    WASP_tmp = []
    first_true=0
    index = []
    for k in range(1, len(precip)):
        # for evary row (ordered month-year combinations):
            # check if droguht month -> calculate drought accumulated magnitude (over 1+ months)
        if drought_class.iloc[k, i]== 1:
            # In case of a drought month.
            # calculate monthly WASP index
            index = drought_class.timing.dt.month[k] - 1
            # WASP monthly index: [(precipitation - month_threshold)/month_threshold)]*[month_threshold/annual_treshold]
            WASP_last=((precip.iloc[k,i] - t_m[index])/t_m[index])* (t_m[index]/t_a)
            
            if first_true==0:
                # if this is the first month in a drought event:
                # append calculated monthly wasp to WASP array.
                WASP_tmp.append(WASP_last)
                first_true=1
            else:
                # if this is NOT the first month in a drought event:
                # add the calculated monthly wasp to last element in the WASP array (accumulative drought).
                WASP_tmp[-1]=WASP_tmp[-1] + WASP_last
            WASP_global.append(WASP_last)
        else:
            # check if not drought month - do not calculate WASP
            first_true=0
    WASP.append(np.array(WASP_tmp))
       

# calculate the exceedance probability from the median global WASP as the Hazard index (dH)

dH = []
WASP = np.array(WASP, dtype=object)

# calculate global median deficit severity - 
    # set drought hazard (dH) as the probability of exceeding the global median water deficit.

median_global_wasp = np.nanmedian(WASP_global)


# calculate dH per region i
for i in range(WASP.shape[0]):
    # The more negative the WASP index, the more severe is the deficit event, so 
    # probability of exceedence the severity is 1 - np.nansum(WASP[i] >= median_global_wasp) / len(WASP[i])
    dH.append(round(1 - np.nansum(WASP[i] >= median_global_wasp) / len(WASP[i]), 3))

output['hazard_raw'] = dH
print('>>>>> Drought hazard is completed.')

Analyzing drought hazard. This process may take few minutes...


Input precipitation data (top 3 rows): 
      timing      CZ05      CZ06      CZ07      CZ08      DE11      DE12  \
0 1979-01-31  0.000414  0.000407  0.000425  0.000440  0.000234  0.000263   
1 1979-02-28  0.000319  0.000359  0.000320  0.000255  0.000327  0.000421   
2 1979-03-31  0.000478  0.000545  0.000328  0.000276  0.000525  0.000576   

       DE13      DE14      DE21  ...      PL62      PL63      HR05      HR06  \
0  0.000349  0.000310  0.000566  ...  0.000704  0.000558  0.000119  0.000412   
1  0.000461  0.000366  0.000443  ...  0.000278  0.000247  0.000102  0.000382   
2  0.000502  0.000473  0.001078  ...  0.000752  0.000449  0.000096  0.000285   

       NO02      NO06      NO08      NO09      NO0A      NO07  
0  0.000549  0.000831  0.000341  0.000918  0.002029  0.002207  
1  0.000896  0.002915  0.000246  0.000511  0.002908  0.004564  
2  0.001620  0.001493  0.001287  0.002428  0.004047  0.002150  

[3 rows x 25

### Save hazard indices

In [1]:
with open("dH.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(dH, f, indent=2) 

print('>>>>> Drought hazard is save as dH.json.')

NameError: name 'json' is not defined

## Conclusions
The above workflow calculates the Drought hazard (dH) index that can be used as an input to calculate drought risk in the workflow described in the file Risk_Assessment.ipynb.

## Contributors
The workflow has beend developed by [Silvia Artuso](https://iiasa.ac.at/staff/silvia-artuso) and [Dor Fridman](https://iiasa.ac.at/staff/dor-fridman) from [IIASA's Water Security Research Group](https://iiasa.ac.at/programs/biodiversity-and-natural-resources-bnr/water-security), and supported by [Michaela Bachmann](https://iiasa.ac.at/staff/michaela-bachmann) from [IIASA's Systemic Risk and Reslience Research Group](https://iiasa.ac.at/programs/advancing-systems-analysis-asa/systemic-risk-and-resilience).

## References

[1] Lyon, B., & Barnston, A. G. (2005). ENSO and the spatial extent of interannual precipitation extremes in tropical land areas. *Journal of climate*, 18(23), 5095-5109.

[2] Carrão, H., Singleton, A., Naumann, G., Barbosa, P., & Vogt, J. V. (2014). An optimized system for the classification of meteorological drought intensity with applications in drought frequency analysis. *Journal of Applied Meteorology and Climatology*, 53(8), 1943-1960.